In [ ]:
from utils import get_clues
from src.opensource.llama_frontend import chat_pipeline
import pandas as pd
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from src.opensource.templates import SYS_LLAMA_TEMPLATE, INST_LLAMA_TEMPLATE, SYS_BLOOMZ_TEMPLATE, INST_BLOOMZ_TEMPLATE

In [ ]:
conversation_buffer = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

In [ ]:
inst_template = PromptTemplate.from_template(INST_LLAMA_TEMPLATE)
inst_template = inst_template.format(state='Jharkhand')
print(inst_template)

In [ ]:
from utils.utils import state_clue_notes_dict
import pandas as pd

In [2]:
p1 = state_clue_notes_dict['jharkhand'][0]

df = pd.read_csv(p1)

FileNotFoundError: [Errno 2] No such file or directory: 'cultural_artifacts/clues/jharkhand/jharkhand_clues_1.csv'

In [ ]:
df_eval = pd.DataFrame(columns=['guess1', 'guess2', 'ground_truth', 'clues'])

In [ ]:
def get_outputs(df, df_eval):
    for i, row in df.iterrows():
        print(f"artifact--{i}---")
        clues = row["clues"].strip().split("\n")
        artifact = row["artifact"].lower().strip()
        output = ""
        for j, clue in enumerate(clues):
            output += f"CLUE-{j+1}: {clue}\n"
            fin_clues = output.strip()

        agent = chat_pipeline(
            clue_list=fin_clues,
            prompt_text=SYS_LLAMA_TEMPLATE,
            conversation_buffer=conversation_buffer,
        )
        guess1 = agent.predict(human_input=inst_template)
        guess1 = guess1.split(":")[1].strip().lower()
        if artifact in guess1:
            df_eval = df_eval.append(
                {
                    "guess1": guess1,
                    "guess2": "NA",
                    "ground_truth": artifact,
                    "clues": row["clues"],
                },
                ignore_index=True,
            )
            conversation_buffer.clear()
            continue
        else:
            guess2 = agent.predict(human_input="Your first guess is not correct")
            guess2 = guess2.split(":")[1].strip().lower()
            df_eval = df_eval.append(
                {
                    "guess1": guess1,
                    "guess2": guess2,
                    "ground_truth": artifact,
                    "clues": row["clues"],
                },
                ignore_index=True,
            )
            conversation_buffer.clear()
    return df_eval

In [ ]:
conversation_buffer.clear()

In [ ]:
df_eval = get_outputs(df_jharkhand, df_eval)

In [ ]:
df_eval.to_csv('jharkhand_evals_with_state_name.csv', index=False)

In [ ]:
df_eval